In [19]:
# Libraries
import pandas as pd
import random
import json
from datasets import load_dataset
import re
import math


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

## Getting instructions

In [3]:
dataset = load_dataset("gsm8k", "main")
dataset["test"][5]

Found cached dataset gsm8k (/Users/iv/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

{'question': 'Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?',
 'answer': 'The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.\nIf every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.\nSo for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.\nAnd for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.\nSo in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.\n#### 64'}

In [4]:
def get_random_test_object(dataset):
    if "test" in dataset and len(dataset["test"]) > 0:
        random_index = random.randint(0, len(dataset["test"]) - 1)
        test_object = dataset["test"][random_index]
        
        # Assuming each object in the dataset has 'question' and 'answer' keys
        question = test_object.get("question", "No question found")
        answer = test_object.get("answer", "No answer found")
        
        return question, answer
    else:
        return None, None

In [5]:
def get_random_mutation(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path, header=None, encoding='utf-8', delimiter='.') 
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1', delimiter='.') 

    random_prompt = random.choice(df[1].tolist())
    return random_prompt


In [6]:
def get_random_mutation_txt(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove any leading/trailing whitespace and filter out empty lines
    prompts = [line.strip() for line in lines if line.strip()]
    
    if prompts:
        return random.choice(prompts)
    else:
        return "No mutation prompts found."

In [7]:
def generate_intructions(question, task_description, max_tokens=100):
    formatted_input = f"{task_description} {question}"
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_new_tokens = max_tokens)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

In [8]:
def process_with_llm(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=1000)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

In [9]:
# write the task description here:
task_description = "Generate an instruction on how to solve the problem, based on the given question "

In [10]:
task_description1 = "Generate an instruction, or advice on how to solve a problem"

#### Hide

In [13]:
# getting mutated prompts
question, answer = get_random_test_object(dataset)

In [14]:
print(question)

Janeth borrowed $2000 and promised to return it with an additional 10% of the amount. If she is going to pay $165 a month for 12 months, how much will be Janeth's remaining balance by then?


In [15]:
print(answer)

Janeth will pay an additional $2000x 10/100 =$<<2000*10/100=200>>200.
So she is going to pay off a total of $2000 + $200 = $<<2000+200=2200>>2200.
Janeth will be able to pay off $165 x 12 = $<<165*12=1980>>1980 in 12 months.
Hence, her remaining balance by then will be $2200 - $1980 = $<<2200-1980=220>>220.
#### 220


## Instruction Generation

In [45]:
num_instructions = 10
generated_instructions = []

for _ in range(num_instructions):
    question, _ = get_random_test_object(dataset)  # Fetch a random question from your dataset
    instruction = generate_intructions(question, task_description)
    generated_instructions.append(instruction)

In [57]:
print(generated_instructions[5])

a) a b) a c) d) a e) a f) a g) a h) a i) a j) a k) a j) a k) a j) a k) a j) a k) a j) a k) a j) a k


#### Test Version 2

In [11]:
# Alternative, that does not include question


def generate_instructions1(num_instructions, max_tokens=1000, temperature=1.5):
    formatted_input = f"Generate a list of {num_instructions} instructions on how to solve a math problem"
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, 
                             max_new_tokens=int(max_tokens),
                             temperature=temperature)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

In [12]:
num_instructions = 10
generated_instructions = []

print(generate_instructions1(num_instructions, temperature=500))

Using a calculator, find the number of squares in the equation.


In [66]:
print(generated_instructions[3])

a) To solve a problem, you need to know how to solve it. b) To solve a problem, you need to know how to solve it. c) To solve a problem, you need to know how to solve it. d) To solve a problem, you need to know how to solve it.


In [13]:
mutation_prompt = get_random_mutation("./Prompt-Engineering-OpenDI/mutation_prompts.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 11, saw 2


In [14]:
mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
print(mutation_prompt)

How would someone with derailment follow this instruction?


In [15]:
def apply_mutation(instruction, mutation_prompt):
    # Example mutation - this can be customized based on your mutation logic
    return f"{mutation_prompt} {instruction}"

mutated_instructions = [apply_mutation(instruction, mutation_prompt) for instruction in generated_instructions]


In [66]:
print(mutated_instructions)

['Modify the following instruction creatively, giving some advice on how to solve it: If Dallas has 21 marbles, then Darla has 21 - 4 = 10 marbles. Mazie gave Darla 10 marbles. The answer: 10.', 'Modify the following instruction creatively, giving some advice on how to solve it: First find the total number of messages Mr. Yu sends: 66 messages / 3 = 66 messages. Then find the total number of messages James delivers to Ms. Thompson: 66 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages / 3 = 66 messages. Then find the total number of messages James delivers to Ms. Thompson: 66 messages + 132 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messa

In [16]:
processed_outputs = [process_with_llm(mutated_instruction) for mutated_instruction in mutated_instructions]

In [69]:
for response in processed_outputs:
    print(response)

Dallas has 21 - 4 = 8 marbles. Mazie gave Darla 8 marbles. The answer: 8.
James delivers to Ms. Thompson: 66 messages + 66 messages = 132 messages. Then find the total number of messages James delivers to Ms. Thompson: 66 messages + 132 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages + 66 messages = 132 messages. Then find the total number of messages Mr. Yu sends: 132 messages 

In [6]:
input_text = "Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Specify the max_new_tokens parameter
outputs = model.generate(input_ids, max_new_tokens=50)  # Adjust the number as needed
print(tokenizer.decode(outputs[0]))


<pad> First find the total cost of the first glass: $5 / glass * 16 glasses = $120. Then find the total cost of the second glass: $120 / glass * 60 glasses = $160. Then find the total cost


In [7]:
input_text = "What was the answer for the previus question I asked?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Specify the max_new_tokens parameter
outputs = model.generate(input_ids, max_new_tokens=50)  # Adjust the number as needed
print(tokenizer.decode(outputs[0]))

<pad> a symphony</s>


In [17]:
def check_answer(answer, output):
    if not (isinstance(answer, str) and isinstance(output, str)):
        raise TypeError("Must be of type str")
    if re.search("\s" + answer +"\s*", output):
        # print("answer is correct")
        return 1
    # print("answer is wrong")
    return 0

In [18]:
def get_logic(answer):
    return re.findall("<<(.*?)>>", answer)

In [20]:
def tokenise_logic(logic):
    tokenised_logic = []
    operators = "+-/*="
    current_token = ""
    
    for c in logic:
        if c in operators:
            tokenised_logic.append(current_token)
            tokenised_logic.append(c)
            current_token = ""
        elif c == '.' or c.isnumeric():
            current_token += c
    tokenised_logic.append(current_token)
    
    return tokenised_logic

In [21]:
def check_logic_sentence(line, logic_tokens):
    i = 0

    for token in logic_tokens:
        if token in line:
            i += 1

    if len(logic_tokens) == i:
        return True
    return False

In [22]:
def fitness(answer, output):
    final_answer = database_answer.split()[-1]
    
    score = 0
    
    logic = get_logic(database_answer)
    
    tokenised_logic_sentences = []
    
    for l in logic:
        tokenised_logic_sentences.append(tokenise_logic(l))
    
    total_score = 1 + len(tokenised_logic_sentences)

    for line in output.split("."):
        for sentence in tokenised_logic_sentences:
            if check_logic_sentence(line, sentence):
                tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                score += 1
                break
    last_line = line
    
    num_answer = 'a'

    for word in last_line.split(" "):
        try:
            num_answer = str(int(word))
        except:
            pass
    
    score += check_answer(final_answer, num_answer)
    return score/total_score

In [32]:
num_instructions = 10
generated_prompts = []
scores = []

for _ in range(num_instructions):
    question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
    instruction = generate_intructions(question, task_description)
    mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
    mutated_instruction = apply_mutation(instruction, mutation_prompt)
    generated_prompts.append([mutated_instruction, database_answer])
    processed_output = process_with_llm(mutated_instruction)
    scores.append(fitness(database_answer, processed_output))

In [33]:
print(scores)

[0.2, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
print(generated_prompts[0][0])

In [31]:
print(generated_prompts[0][1])

The cheaper cards are worth a total of 30*50=$<<30*50=1500>>1500
So all the cards together are worth 1500+4000+1000=$<<1500+4000+1000=6500>>6500
The cost for the cards was 1800*3=$<<1800*3=5400>>5400
So he made a profit of 6500-5400=$<<6500-5400=1100>>1100
#### 1100
